# Implementar un MLP con PyTorch para clasificación basado en el dataset de agresividad

<img src="figs/fig-MLP_XOR.png" width="50%">


1. **Definir los preprocesamientos para el texto**:  
   - convertir a minúsculas
   - normalizar el texto: borrar símbolos, puntuación, caracteres duplicados, etc.

2. **Separar los datos para entrenamiento y prueba**:  
   - Crear los dataset de entrenamiento y test con al función train_test_split 

3. **Construir la matriz de Documento-Término**:  
   - Definir los parámetros para usar unigramas
   - Usar la clase TfidfVectorizer para construir la matriz con los datos de entrenamiento

   
4. **Preparar los lotes de datos (minibatches) para el entrenamiento de la red**:  
   - Definir los minibatches con la matriz TFIDF construida

5. **Definir la arquitectura de la red**:  
   - Definir entradas, salidas,  capas de la red y funciones de activación

6. **Entrenar el modelo**:  
   - Definir los parámetros de las red como: número de épocas, learning_rate, número de neuronas para las capas ocultas, etc.
   
7. **Evaluar el modelo**:  
   - Después del entrenamiento, probar la red con las entradas del conjunto de test y evaluar el desempeño con las métricas: Precisión, Recall, F1-score o F1-Measure y Accuracy.
   


# Definición de los datos y minibatches

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split


# colocar la semilla para la generación de números aleatorios para la reproducibilidad de experimentos
random_state = 42
torch.manual_seed(random_state)
np.random.seed(random_state)



# TODO: Definir las funciones de preprocesamiento de texto vinculadas al proceso de creación de la matriz 
# Documeno-Término creada con TfidfVectorizer.


# TODO: Codificar las etiquetas de los datos a una forma categórica numérica: LabelEncoder.


# TODO: Dividir el conjunto de datos en conjunto de entrenamiento (80%) y conjunto de pruebas (20%)


# TODO: Crear la matriz Documento-Término con el dataset de entrenamiento: tfidfVectorizer


# Crear minibatches en PyTorch usando DataLoader
def create_minibatches(X, Y, batch_size):
    # Recibe los documentos en X y las etiquetas en Y
    dataset = TensorDataset(X, Y) # Cargar los datos en un dataset de tensores
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader


# Definición de la arquitectura de la red

In [4]:

# Definir la red neuronal en PyTorch heredando de la clase base de Redes Neuronales: Module
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        # Definición de capas, funciones de activación e inicialización de pesos
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)

        if self.fc1.bias is not None:
            nn.init.zeros_(self.fc1.bias)
        if self.fc2.bias is not None:
            nn.init.zeros_(self.fc2.bias)        

    
    def forward(self, X):
        # Definición del orden de conexión de las capas y aplición de las funciones de activación
        out = self.fc1(X)
        out = self.sigmoid(out)  # Aplicamos la sigmoide en la capa oculta
        out = self.fc2(out)
        out = self.sigmoid(out)  # Aplicamos la sigmoide en la capa de salida
        return out

# Entrenamiento de la red

In [ ]:
# Establecer los parámetros de la red

# Parámetros de la red
input_size = 0 # ?
hidden_size = 0 # ? 
output_size = 1  # La salida es el resultado de la sigmoide para un clasificador binario: 0 o 1
epochs = 1000    # variar el número de épocas, para probar que funciona la programación solo usar 2 épocas
learning_rate = 0.2
# Se recomiendan tamaños de batch_size potencias de 2: 16, 32, 64, 128, 256
# Entre mayor el número más cantidad de memoria se requiere para el procesamiento
batch_size = 64 # definir el tamaño del lote de procesamiento 


# TODO: Convertir los datos de entrenamiento y etiquetas a tensores  de PyTorch

X_train = [] # ?
Y_train = [] # ?

# Crear la red
model = MLP(input_size, hidden_size, output_size)

# Definir la función de pérdida
# Mean Square Error (MSE)
criterion = nn.MSELoss()

# Definir el optimizador
#Parámetros del optimizador: parámetros del modelo y learning rate 
# Stochastic Gradient Descent (SGD)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Entrenamiento
print("Iniciando entrenamiento en PyTorch")

# Poner el modelo en modo de entrenamiento
model.train()  

for epoch in range(epochs):
    lossTotal = 0
    #definir el batch_size
    dataloader = create_minibatches(X_train, Y_train, batch_size=batch_size)
    for X_tr, y_tr in dataloader:
        # inicializar los gradientes en cero para cada época
        optimizer.zero_grad()
        
        # Propagación hacia adelante
        y_pred = model(X_tr)  #invoca al método forward de la clase MLP
        
        # Calcular el error MSE
        loss = criterion(y_pred, y_tr)
        #Acumular el error 
        lossTotal += loss.item()
        
        # Propagación hacia atrás: cálculo de los gradientes de los pesos y bias
        loss.backward()
        
        # actualización de los pesos: regla de actualización basado en el gradiente W = W - learning_rate * dE/dW
        optimizer.step()

    print(f"Época {epoch+1}/{epochs}, Pérdida: {lossTotal/len(dataloader)}")


### Modo para predicción de datos

In [ ]:
# TODO: Transformar el dataset de test con los mismos preprocesamientos y al  espacio de representación vectorial que el modelo entrenado, es decir, 
# Al espacio de la matriz TFIDF

# Convertir los datos de prueba a tensores de PyTorch

X_test = [] # ?


# Desactivar el comportamiento de modo de  entrenamiento: por ejemplo, capas como Dropout
model.eval()  # Establecer el modo del modelo a "evaluación"

with torch.no_grad():  # No  calcular gradientes 
    y_pred_test = model(X_test)

# y_test_pred contiene las predicciones

# Obtener la clase real
y_pred_test2 = torch.where(y_pred_test>=0.5, 1, 0)

print(y_pred_test2)


### Evaluación

In [ ]:
# TODO: Evaluar el modelo con las predicciones obtenidas y las etiquetas esperadas: classification_report y  matriz de confusión (métricas Precisión, Recall, F1-measaure, Accuracy)
